In [ ]:
%matplotlib inline

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt

def get_last_days(dates, ref_date, num):
    last_dates = []
    # linear search right now
    ind = len(dates)
    for i in range(len(dates)):
        if dates[i] >= ref_date:
            ind = i
            break
    return dates[(ind - num) : (ind - 1)]
    
    
SEC = 1000000
MIN = 60 * SEC
DOLLAR = 1000000
DATA_PATH = "output/"

ticker = "IVV"
ref_date = "20150204"

files = os.listdir(DATA_PATH)
dates = []
for f in files:
    if f.endswith("_.txt"):
        dates.append(f.split('_')[1])
dates.sort()

last_dates = get_last_days(dates, ref_date, 15)
last_dates_dt = [dt.datetime.strptime(x, "%Y%m%d").date() for x in last_dates]

output = np.zeros(0)
sample_times = []
for date in last_dates:
    output_filename = "{0}{1}_{2}_output.txt".format(DATA_PATH, ticker, date)
    if output.shape[0] == 0:
        output = np.loadtxt(output_filename)
        sample_times = output[:, 0].tolist()
    else:
        temp = np.loadtxt(output_filename)
        output = np.concatenate((output, temp), axis=0)

# Visualize PNL

pnl = output[output[:, 0] == sample_time[-1], 1]
print len(pnl) == len(last_dates_dt)

fig, ax = plt.subplots()
ax.plot(last_dates_dt, pnl)
ax.set(title="Daily PNL", ylabel="PNL", xlabel = "Date")
plt.show()
# plt.savefig("{0}{1}_{2}_{3}_pnl.png".format(DATA_PATH, ticker, last_dates[0], last_dates[-1]), dpi = 300)

# Visualize inventory

inventory = []
for time in sample_times:
    inventory.append(np.mean(output[output[:, 0] == time, 2]))

fig, ax = plt.subplots()
ax.plot(last_dates_dt, inventory)
ax.set(title="Inventory", ylabel="Stocks", xlabel = "Date")
plt.show()
# plt.savefig("{0}{1}_{2}_{3}_inventory.png".format(DATA_PATH, ticker, last_dates[0], last_dates[-1]), dpi = 300)
